In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("./storage/Kaggle_M5_Luke/sales_train_validation.csv")
df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data) 
print(time_series_data.shape)

(30490, 1913)


In [5]:
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps-1): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

x_train, y_train = split_xy2(time_series_data, 7) 

x_train = x_train.reshape(1905,7,30490) 

x_val = time_series_data[:,1905:1912] 
y_val = time_series_data[:,1912] 
x_val = x_val.reshape(1,7,30490) 
y_val = y_val.reshape(1,30490)

print(x_train.shape) 
print(y_train.shape)
print(x_val.shape) 
print(y_val.shape)
    

(1905, 7, 30490)
(1905, 30490)
(1, 7, 30490)
(1, 30490)


In [6]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(time_steps,features),activation='relu')) 
    model.add(Dense(30490)) 
    return model 
model = build_model(7,30490)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8)                 975968    
_________________________________________________________________
dense_1 (Dense)              (None, 30490)             274410    
Total params: 1,250,378
Trainable params: 1,250,378
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='mse',optimizer='adam',metrics=['mse'])
model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base2/model{epoch:02d}.h5',period=50)
model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  


Train on 1905 samples, validate on 1 samples
Epoch 1/1000
1905/1905 [==============================] - 4s 2ms/step - loss: 11.7697 - mse: 11.7697 - val_loss: 7.3285 - val_mse: 7.3285
Epoch 2/1000
1905/1905 [==============================] - 4s 2ms/step - loss: 7.6663 - mse: 7.6663 - val_loss: 6.9498 - val_mse: 6.9498
Epoch 3/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 7.3192 - mse: 7.3192 - val_loss: 7.1213 - val_mse: 7.1213
Epoch 4/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 7.2412 - mse: 7.2412 - val_loss: 7.0847 - val_mse: 7.0847
Epoch 5/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 7.2064 - mse: 7.2064 - val_loss: 7.1347 - val_mse: 7.1347
Epoch 6/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 7.1880 - mse: 7.1880 - val_loss: 7.0738 - val_mse: 7.0738
Epoch 7/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 7.1681 - mse: 7.1681 - val_loss: 7.1753 - val_mse: 7.1753
E

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.108725). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1905/1905 [==============================] - 3s 2ms/step - loss: 6.7769 - mse: 6.7769 - val_loss: 7.1813 - val_mse: 7.1813
Epoch 479/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7775 - mse: 6.7775 - val_loss: 7.1597 - val_mse: 7.1597
Epoch 480/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7798 - mse: 6.7798 - val_loss: 7.2841 - val_mse: 7.2841
Epoch 481/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7810 - mse: 6.7810 - val_loss: 7.1198 - val_mse: 7.1198
Epoch 482/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7782 - mse: 6.7782 - val_loss: 7.0842 - val_mse: 7.0842
Epoch 483/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7749 - mse: 6.7749 - val_loss: 7.1825 - val_mse: 7.1825
Epoch 484/1000
1905/1905 [==============================] - 3s 2ms/step - loss: 6.7728 - mse: 6.7728 - val_loss: 7.2948 - val_mse: 7.2948
Epoch 485/1000
1905/1905 [=======================